In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../real_monica/data.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(46, 47), dtype=bool)

In [6]:
motion.shape

(8620, 48)

In [7]:
motion[:50, 1]

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2])

In [8]:
contacts.shape

(8620, 2)

In [9]:
contacts[0:5]

array([[False, False],
       [ True, False],
       [False,  True],
       [ True, False],
       [False,  True]])

In [10]:
motion[0]

array([ 1.67561612e+09,  1.00000000e-01,  0.00000000e+00,  0.00000000e+00,
        2.20218942e-01,  1.28156364e-01, -3.79664570e-01,  2.50404894e-01,
       -1.29626960e-01, -3.77603322e-01, -2.49196276e-01,  1.29706442e-01,
       -3.83550644e-01, -2.75959224e-01, -1.30796552e-01, -3.82629216e-01,
        2.70977259e+00,  1.16201282e+00,  3.74220192e-01,  8.41218606e-02,
       -4.70607728e-03,  8.78726691e-02,  4.04834114e-02, -1.11447081e-01,
       -4.68525977e-04, -1.83918811e-02, -8.77368897e-02,  4.61280271e-02,
        1.03009783e-01,  1.05129434e-02,  4.17001754e-01,  4.04417783e-01,
       -3.86051983e-02,  6.37666732e-02, -6.84953667e-03,  1.16000000e+02,
        6.40000000e+01,  8.70000000e+01,  7.40000000e+01, -8.40414374e-04,
       -5.61846653e-04,  9.31148469e-01, -3.64638895e-01, -4.33430600e-04,
        1.97484461e-03, -2.39510274e+00,             nan,             nan])

## Height-Force plots

In [11]:
def get_specific_cmd(dataset, fwd, side, rot):
    if abs(fwd): 
        return np.where(dataset[:, 1] > 0)[0] if fwd > 0 else np.where(dataset[:, 1] < 0)[0]
    if abs(side): 
        return np.where(dataset[:, 2] > 0)[0] if side > 0 else np.where(dataset[:, 2] < 0)[0]
    if abs(rot): 
        return np.where(dataset[:, 3] > 0)[0] if rot > 0 else np.where(dataset[:, 3] < 0)[0]

In [12]:
def get_swinging_motions(dataset, height=1):
    # rf min height (i.e swining motion)
    fl_min_height = np.where(dataset[:, 6] < height)[0]
    fr_min_height = np.where(dataset[:, 9] < height)[0]
    rl_min_height = np.where(dataset[:, 12] < height)[0]
    rr_min_height = np.where(dataset[:, 15] < height)[0]
            
    return fl_min_height,fr_min_height,rl_min_height,rr_min_height

# Dataset Preparation

In [13]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [14]:
def yaw_from_quaternion(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    x = Q[0]
    y = Q[1]
    z = Q[2]
    w = Q[3]
     
    return np.arctan2(2 * (w*z + x*y), 1 - 2 * (y*y + z*z))

In [15]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [16]:
def create_acceleration_com_dataset(dataset, footsteps, motion=None, debug=False):
    idx = 1
    inputs = []
    labels = []
    
    while idx < len(footsteps):
        if idx > 0:
            # Compute time difference footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            # Round velocity array
            dataset[footsteps[idx-1], 1:4] = np.around(dataset[footsteps[idx-1], 1:4], decimals=1)
            dataset[footsteps[idx], 1:4] = np.around(dataset[footsteps[idx], 1:4], decimals=1)
            
            if time_difference < 0.4:
                fl_rr_moving = contacts[footsteps[idx], 0]
                fr_rl_moving = contacts[footsteps[idx], 1]

                if fl_rr_moving == fr_rl_moving:
                    print("Invalid footstep")
                    idx += 1
                    continue

                # Rotation matrices
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 39:43])
                #print(R_curr)

                # Retrieve base poses in world frame
                prev_base = dataset[footsteps[idx-1], 16:19]
                curr_base = dataset[footsteps[idx], 16:19]
                
                # Compute base displacement
                world_displacement = curr_base - prev_base
                base_displacement = np.dot(R_curr.T, world_displacement)
                
                #print(dataset[footsteps[idx], 39:43])
                #print(dataset[footsteps[idx], 1], curr_base[0], prev_base[0], world_displacement[0], base_displacement[0])
                
                # Compute yaw displacement
                prev_yaw = yaw_from_quaternion(dataset[footsteps[idx-1], 39:43])
                curr_yaw = yaw_from_quaternion(dataset[footsteps[idx], 39:43])
                yaw = curr_yaw - prev_yaw
                
                #print(dataset[footsteps[idx-1], 19], ",", dataset[footsteps[idx], 1], ",", dataset[footsteps[idx], 19])
                #print("\n")
                
                #print(world_displacement[0], base_displacement[0])
                
                inputs.append(dataset[footsteps[idx-2], 1:2].tolist() +
                              dataset[footsteps[idx-1], 1:2].tolist() +
                              dataset[footsteps[idx], 1:2].tolist() +
                              dataset[footsteps[idx-2], 19:20].tolist() +
                              dataset[footsteps[idx-1], 19:20].tolist() +
                              dataset[footsteps[idx-1], 4:16].tolist() +
                              [fl_rr_moving, fr_rl_moving]) # swiging booleans
                
                labels.append([base_displacement[0], base_displacement[1], yaw])
        idx += 1
                
    return np.array(inputs, dtype=object), np.array(labels, dtype=object)

# Stack datasets
X_com, Y_com = create_acceleration_com_dataset(motion, [x for x in range(len(motion))])
print(X_com.shape)
print(Y_com.shape)

(8592, 19)
(8592, 3)


In [17]:
%store X_com
%store Y_com

Stored 'X_com' (ndarray)
Stored 'Y_com' (ndarray)


In [40]:
X_com[0, :]

array([0.0, 0.1, 0.1, -0.0124955084175, 0.0841218605638, 0.220218941569,
       0.128156363964, -0.379664570093, 0.250404894352, -0.129626959562,
       -0.377603322268, -0.249196276069, 0.129706442356, -0.383550643921,
       -0.275959223509, -0.130796551704, -0.382629215717, True, False],
      dtype=object)

In [41]:
dic = dict()
for x in range(len(X_com)):
    key = str(round(X_com[x, 0], 1)) + str(round(X_com[x, 3], 3))
    if key in dic:
        dic[key][0].append(Y_com[x, 0])
        dic[key][1].append(X_com[x, 4])
        dic[key][2].append([X_com[x, 7], 
                            X_com[x, 10], 
                            X_com[x, 13], 
                            X_com[x, 16]])
        dic[key][3].append(X_com[x, -1])
    else:
        dic[key] = [[Y_com[x, 0]], 
                    [X_com[x, 4]],
                    [[X_com[x, 7], 
                     X_com[x, 10], 
                     X_com[x, 13], 
                     X_com[x, 16]]],
                     [X_com[x, -1]]]

In [42]:
#dic.keys()

In [48]:
for key in dic.keys():
    #print(f"{key[:3]} -> {key[3:]}. Mean: {np.round(np.mean(dic[key][0]), 4)}. Std: {np.round(np.std(dic[key][0]), 3)}.")
    if float(key[:3]) == 1.0 and np.isclose(float(key[3:]), 1.0, 0.05):
        for x in range(len(dic[key][0])):
            print(f"Vact: {dic[key][1][x]}. CoM Disp: {dic[key][0][x]}")

Vact: 1.02865719795. CoM Disp: 0.3067635949574626
Vact: 1.03109622002. CoM Disp: 0.2885332150843747
Vact: 1.00633144379. CoM Disp: 0.2798533483959944
Vact: 1.00761842728. CoM Disp: 0.2857465132599348
Vact: 1.10759794712. CoM Disp: 0.2903413070342029
Vact: 1.02620697021. CoM Disp: 0.27288903811533066
Vact: 1.02086162567. CoM Disp: 0.282591046641069
Vact: 1.20186626911. CoM Disp: 0.3088262602002606
Vact: 1.02343225479. CoM Disp: 0.2750476026848501
Vact: 0.970872282982. CoM Disp: 0.2728850667373172
Vact: 1.22101485729. CoM Disp: 0.2954004253549713
Vact: 1.00214087963. CoM Disp: 0.29354807794397125
Vact: 1.16557645798. CoM Disp: 0.30881782814874115
Vact: 1.19213378429. CoM Disp: 0.3277683426188527
Vact: 1.05347049236. CoM Disp: 0.30324031788302547
Vact: 1.00575757027. CoM Disp: 0.2824735298468723
Vact: 1.0302220583. CoM Disp: 0.2404869573496881
Vact: 0.99085187912. CoM Disp: 0.2836383847323799
Vact: 1.07011497021. CoM Disp: 0.2808731722595267
Vact: 0.708061814308. CoM Disp: 0.1015671336764

In [44]:
for key in dic.keys():
    for x in range(len(dic[key][0])):
        print(f"{key[:3]}->{key[3:]}. Disp: {np.round(dic[key][0][x], 6)}. CoM Vel: {np.round(dic[key][1][x], 3)}.")

0.0->-0.012. Disp: 0.028651. CoM Vel: 0.084.
0.0->-0.012. Disp: 0.002907. CoM Vel: 0.015.
0.0->-0.012. Disp: 0.064928. CoM Vel: 0.133.
0.0->-0.012. Disp: 0.001328. CoM Vel: 0.029.
0.0->-0.012. Disp: -0.003117. CoM Vel: 0.017.
0.0->-0.012. Disp: -0.002922. CoM Vel: 0.034.
0.0->-0.012. Disp: 0.002591. CoM Vel: -0.01.
0.0->-0.012. Disp: 0.136596. CoM Vel: 0.246.
0.0->-0.012. Disp: 0.001055. CoM Vel: -0.031.
0.0->-0.012. Disp: -0.000159. CoM Vel: 0.015.
0.0->-0.012. Disp: -1.6e-05. CoM Vel: 0.025.
0.0->-0.012. Disp: -0.000605. CoM Vel: -0.024.
0.0->-0.012. Disp: 0.001542. CoM Vel: 0.004.
0.0->-0.012. Disp: -0.000811. CoM Vel: -0.027.
0.0->-0.012. Disp: -0.005205. CoM Vel: 0.004.
0.0->-0.012. Disp: -0.001849. CoM Vel: -0.029.
0.0->-0.012. Disp: 0.002163. CoM Vel: 0.024.
0.0->-0.012. Disp: 0.008715. CoM Vel: -0.002.
0.0->-0.012. Disp: -0.006302. CoM Vel: 0.025.
0.0->-0.012. Disp: -0.000974. CoM Vel: -0.004.
0.0->-0.012. Disp: 0.000862. CoM Vel: 0.009.
0.0->-0.012. Disp: -0.00087. CoM Vel: -0

In [45]:
for key in dic.keys():
    print(f"{key[:3]}->{key[3:]}. Mean: {np.round(np.mean(dic[key][0]), 3)}. Std: {np.round(np.std(dic[key][0]), 3)}.")

0.0->-0.012. Mean: 0.009. Std: 0.029.
0.1->0.084. Mean: 0.026. Std: 0.005.
0.1->0.116. Mean: 0.029. Std: 0.003.
0.1->0.106. Mean: 0.029. Std: 0.002.
0.1->0.104. Mean: 0.038. Std: 0.046.
0.1->0.092. Mean: 0.029. Std: 0.001.
0.1->0.095. Mean: 0.019. Std: 0.007.
0.1->0.077. Mean: 0.031. Std: 0.0.
0.1->0.098. Mean: 0.036. Std: 0.009.
0.1->0.11. Mean: 0.022. Std: 0.006.
0.1->0.108. Mean: 0.021. Std: 0.013.
0.1->0.115. Mean: 0.022. Std: 0.003.
0.1->0.097. Mean: 0.015. Std: 0.019.
0.1->0.099. Mean: 0.028. Std: 0.001.
0.1->0.105. Mean: 0.02. Std: 0.01.
0.1->0.087. Mean: 0.026. Std: 0.005.
0.1->0.111. Mean: 0.017. Std: 0.007.
0.1->0.114. Mean: 0.048. Std: 0.031.
0.1->0.083. Mean: 0.035. Std: 0.015.
0.1->0.118. Mean: 0.083. Std: 0.055.
0.1->0.075. Mean: 0.022. Std: 0.008.
0.1->0.088. Mean: 0.029. Std: 0.002.
0.1->0.085. Mean: 0.023. Std: 0.002.
0.1->0.096. Mean: 0.025. Std: 0.0.
0.1->0.101. Mean: 0.045. Std: 0.046.
0.1->0.081. Mean: 0.033. Std: 0.002.
0.1->0.121. Mean: 0.029. Std: 0.0.
0.1->0.09